In [2]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os, requests
from astropy.time import Time
from astropy.table import Table
from astropy.coordinates import SkyCoord
# from style import output_folder, big_fontsize, base_width, base_height, dpi
import seaborn as sns
import json
from astropy.time import Time

In [3]:
notice_summary_urls = {
    'BRONZE_GOLD': "https://gcn.gsfc.nasa.gov/amon_icecube_gold_bronze_events.html",
    'EHE': "https://gcn.gsfc.nasa.gov/amon_ehe_events.html",
    'HESE': "https://gcn.gsfc.nasa.gov/amon_hese_events.html"
}


def get_summary_table(k, renew=False):
    fn = f"data/gcn_notice_summary_{k}.html"
    if not os.path.isfile(fn) or renew:
        _t = pd.read_html(requests.get(notice_summary_urls[k]).text)[0]
        _t.to_html(fn)
    else:
        _t = pd.read_html(fn, index_col=0)[0]
    return _t

In [4]:
def parse_notice_info(ic_name, alert_class, verbose=True):
    
    if ic_name in ['IC210503A', 'IC200107A', 'IC210717A']:
        if verbose: print(f"{ic_name}: No notice becasue selected offline")
        return None, None, None, None
    
    if ('BRONZE' in alert_class) or ('GOLD' in alert_class):
        _alert_class = 'BRONZE_GOLD'
    else:
        _alert_class = alert_class
        
    _pos_ext = ' [deg]' if _alert_class == 'BRONZE_GOLD' else ''
    _error_ext = '90 [arcmin]' if _alert_class == 'BRONZE_GOLD' else ''
        
    summary_table = get_summary_table(_alert_class)
    
    _date_from_name = ic_name.split('IC')[-1][:-1]
    _dates_in_table = summary_table['EVENT', 'Date'].apply(lambda x: x.replace('/', ''))
    _mask = _dates_in_table == _date_from_name
    
    if 'Rev' in summary_table['EVENT'].columns:
        _mask = _mask & (summary_table['EVENT', 'Rev'] == 0)
    
    _selected = summary_table[_mask]
    
    if len(_selected) != 1:
        
        if 'IC160427A' in ic_name:
            if verbose:
                print(f'{ic_name}: selecting the third notice of {len(_selected)}')
            _ind = 1
            
        elif len(_selected) == 2:
            
            if verbose:
                print(f"{ic_name}: Two matching dates.")
                
            _ras = _selected["OBSERVATION"][f"RA{_pos_ext}"]
            _decs = _selected["OBSERVATION"][f"Dec{_pos_ext}"]
            _coords = SkyCoord(_ras, _decs, unit='deg')
            _sep = _coords[0].separation(_coords[1]).deg
            if verbose: print(f"\t{_sep:.2f} degrees apart")
                
            if _sep > 1:
                if verbose: print(f"\tassuming it's two alerts at the same day")
                dates = [d.replace('/','-') for d in _selected['EVENT', 'Date']]
                tstrings = np.array([f"20{_s['EVENT', 'Date'].replace('/','-')}T{_s['EVENT', 'Time UT']}"
                           for _, _s in _selected.iterrows()])
                times = Time(tstrings)
                _ind = np.argmin(times) if ic_name.endswith('A') else np.argmax(times) if ic_name.endswith('B') else None
                
            else: 
                if verbose: print(f"\tassuming second notice is refined info from circular. choosing first one")
                _ind = 0
                
        else:
            raise Exception(f"More than one entry for {ic_name}: {_selected}")
            
    else:
        _ind = 0
        
    _selected = _selected.iloc[_ind]
    _date = _selected["EVENT"]["Date"].replace("/", "-")
    _obstime = _selected["EVENT"]["Time UT"]
    _ra = _selected["OBSERVATION"][f"RA{_pos_ext}"]
    _dec = _selected["OBSERVATION"][f"Dec{_pos_ext}"]
    _error90 = _selected["OBSERVATION"][f"Error{_error_ext}"]
    
    _arrivaltime = f"20{_date} {_obstime}"
    
    return _arrivaltime, _ra, _dec, _error90
    

In [5]:
obs = pd.read_csv("data/nu_alerts_observed.csv", skiprows=[0, 1, 2])
obs = obs[~np.isnan(obs["RA"])]
non = pd.read_csv("data/nu_alerts_unobserved.csv", skiprows=[0, 1], usecols=range(11))
comb = pd.concat([obs, non], ignore_index=True)

# Splitting the EHE and HESE info into two rows
m = comb['Event'] == 'IC160731A'
comb.loc[m, 'Class'] = 'EHE'
to_append = comb.loc[m].copy()
to_append['Class'] = 'HESE'
comb = comb.append(to_append)

new_cols = ['arrival time [UT]','initial RA','initial Dec','initial Error90 [arcmin]']
for c in new_cols:
    comb[c] = np.nan

for j, (i, row) in enumerate(comb.iterrows()):
    m = (comb.Event == row.Event) & (comb.Class == row.Class)
    comb.loc[m, new_cols] = parse_notice_info(row['Event'], row['Class'])
    
comb

IC190922B: Two matching dates.
	148.09 degrees apart
	assuming it's two alerts at the same day
IC200107A: No notice becasue selected offline
IC200926A: Two matching dates.
	94.11 degrees apart
	assuming it's two alerts at the same day
IC160427A: selecting the third notice of 4
IC160731A: Two matching dates.
	0.56 degrees apart
	assuming second notice is refined info from circular. choosing first one
IC161103A: Two matching dates.
	0.07 degrees apart
	assuming second notice is refined info from circular. choosing first one
IC161210A: Two matching dates.
	1.07 degrees apart
	assuming it's two alerts at the same day
IC190922A: Two matching dates.
	148.09 degrees apart
	assuming it's two alerts at the same day
IC200926B: Two matching dates.
	94.11 degrees apart
	assuming it's two alerts at the same day
IC201115A: Two matching dates.
	49.55 degrees apart
	assuming it's two alerts at the same day
IC201115B: Two matching dates.
	49.55 degrees apart
	assuming it's two alerts at the same day
IC

,Event,Class,RA,RA Unc (rectangle),Dec,Dec Unc (rectangle),Area (rectangle),Observed area (from Healpix),Observed area (corrected for chip gaps),Signalness,...,ZTF ATEL/GCN,Additional ZTF GCN,Additional ZTF GCN.1,Rejection reason,Code,Unnamed: 10,arrival time [UT],initial RA,initial Dec,initial Error90 [arcmin]
0,IC190503A,EHE,120.28,"[0.57, -0.77]",6.35,"[0.76, -0.7]",1.944284,1.642524,1.37,0.36000,...,http://www.astronomerstelegram.org/?read=12730,NaN,NaN,NaN,NaN,NaN,2019-05-03 17:23:08.72,120.3040,6.3568,14.99
1,IC190619A,GOLD,343.26,"[4.08, -2.63]",10.73,"[1.51, -2.61]",27.209992,25.732874,21.57,0.55000,...,http://www.astronomerstelegram.org/?read=12879,NaN,NaN,NaN,NaN,NaN,2019-06-19 13:14:18.04,342.7772,10.0547,40.28
2,IC190730A,GOLD,225.79,"[1.28, -1.43]",10.47,"[1.14, -0.89]",5.407511,4.960357,4.52,0.67000,...,http://www.astronomerstelegram.org/?read=12974,NaN,NaN,NaN,NaN,NaN,2019-07-30 20:50:41.31,226.8302,10.5078,45.31
3,IC190922B,GOLD,5.76,"[1.19, -1.37]",-1.57,"[0.93, -0.82]",4.478434,4.114506,4.09,0.51000,...,https://gcn.gsfc.nasa.gov/gcn3/25824.gcn3,NaN,NaN,NaN,NaN,NaN,2019-09-22 23:03:55.56,6.0049,-1.4881,38.02
4,IC191001A,GOLD,314.08,"[6.56, -2.26]",12.94,"[1.5, -1.47]",25.528634,28.651031,23.06,0.59000,...,https://gcn.gsfc.nasa.gov/gcn3/25929.gcn3,NaN,NaN,NaN,NaN,NaN,2019-10-01 20:09:18.17,314.3550,12.5755,47.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,IC210519A,BRONZE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Alert retraction,1.0,NaN,2021-05-19 20:40:02.93,167.6902,-25.8725,129.06
82,IC210608A,BRONZE,337.41,"[4.89, -11.64]",18.37,"[3.75, -3.24]",NaN,NaN,NaN,0.31467,...,NaN,NaN,NaN,Poor Signalness and Localization,4.0,NaN,2021-06-08 03:41:00.97,337.0182,18.6470,78.35
83,IC210717A,EXTRA,46.49,"[2.40, -2.57]",-1.34,"[2.63, -3.41]",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Poor Signalness and Localization,4.0,NaN,None,NaN,NaN,NaN
84,IC210730A,BRONZE,105.73,"[2.00, -1.85]",14.79,"[0.91, -0.86]",NaN,NaN,NaN,0.31923,...,NaN,NaN,NaN,Separation from galactic plane,8.0,NaN,2021-07-30 22:12:40.62,105.8360,14.8168,30.80


In [6]:
comb['retracted'] = (comb['Rejection reason'] == 'Alert retraction') | (comb['Rejection reason'] == 'Alert Retraction')

In [7]:
keep_cols = ['Event', 'Class', 'RA', 'RA Unc (rectangle)', 'Dec',
       'Dec Unc (rectangle)', 'arrival time [UT]',
       'initial RA', 'initial Dec', 'initial Error90 [arcmin]', 'retracted']
out = comb[keep_cols].sort_values('Event')
out

,Event,Class,RA,RA Unc (rectangle),Dec,Dec Unc (rectangle),arrival time [UT],initial RA,initial Dec,initial Error90 [arcmin],retracted
23,IC160427A,HESE,240.57,"[0.6, -0.6]",9.34,"[0.6, -0.6]",2016-04-27 05:52:32.00,239.6639,6.8528,534.00,False
24,IC160731A,EHE,214.50,"[0.75, -0.75]",-0.33,"[0.75, -0.75]",2016-07-31 01:55:04.00,214.5440,-0.3347,20.99,False
24,IC160731A,HESE,214.50,"[0.75, -0.75]",-0.33,"[0.75, -0.75]",2016-07-31 01:55:04.00,215.1090,-0.4581,73.79,False
25,IC160806A,EHE,122.81,"[0.5, 0.5]",-0.81,"[0.5, -0.5]",2016-08-06 12:21:33.00,122.7980,-0.7331,6.67,False
26,IC160814A,HESE,200.30,"[2.43, -3.03]",-32.40,"[1.39, -1.21]",2016-08-14 21:45:54.00,199.3100,-32.0165,89.39,False
...,...,...,...,...,...,...,...,...,...,...,...
82,IC210608A,BRONZE,337.41,"[4.89, -11.64]",18.37,"[3.75, -3.24]",2021-06-08 03:41:00.97,337.0182,18.6470,78.35,False
21,IC210629A,BRONZE,340.75,"[1.11, -2.23]",12.94,"[0.91, -0.93]",2021-06-29 18:09:44.22,340.6350,12.6111,30.80,False
83,IC210717A,EXTRA,46.49,"[2.40, -2.57]",-1.34,"[2.63, -3.41]",None,NaN,NaN,NaN,False
84,IC210730A,BRONZE,105.73,"[2.00, -1.85]",14.79,"[0.91, -0.86]",2021-07-30 22:12:40.62,105.8360,14.8168,30.80,False


In [8]:
out.to_csv('data/ASASSN_sample_paper_IceCube_info.csv')